In [ ]:
import requests
import warnings
import pandas as pd
import datetime
from pandas import to_datetime
import time
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from tqdm import tqdm
from joblib import Parallel, delayed

import urllib
import pytz
import json
from itertools import chain

pd.set_option('display.float_format', lambda x: '%.6f' % x)
warnings.filterwarnings("ignore")

In [ ]:
p2p_grpc_node = "34.94.26.165:9090"
#p2p_rpc_node = "https://rpc-1-regen.tm.p2p.org"  # "https://rpc-1.regen.p2p.org" - old - 300 $ per month
p2p_rpc_node = "https://rpc-regen.ecostake.com"
#p2p_lcd_node = "https://lcd-1-regen.tm.p2p.org"
p2p_lcd_node = "http://public-rpc.regen.vitwit.com:1317"


#Inputs
report_date = '2024-08-01' #'2023-05-10'
start_date = '2024-04-01'
end_date = '2024-07-31'

start_date = pd.to_datetime(start_date).replace(tzinfo=pytz.UTC)
end_date = pd.to_datetime(end_date).replace(tzinfo=pytz.UTC)


In [ ]:

#utility function for queries
def cosmos_paginated_query(url):
    result = []
    is_last_page = False
    key = ''
    save_key = ''
    while key != '?pagination.key=None':
        url_key = url+key
        res = requests.get(url_key).json()
        try:
            key = '?pagination.key=' + urllib.parse.quote_plus(str(res['pagination']['next_key']))
        except:
            print('something went wrong with url: ', url+key)
        if save_key == key:
            break
        save_key = key
        result.append(res)
        print('Okay')
        time.sleep(1)
    return result

def get_block_time(address, height='last'):
    if height == 'last':
        req = requests.get(address+'/block').json()
    else:
        req = requests.get(address+'/block?height='+str(height)).json()
    try:
        ts = req['result']['block']['header']['time']
        height = req['result']['block']['header']['height']
    except KeyError:
        ts = req['block']['header']['time']
        height = req['block']['header']['height']
    return ts,height

def get_new_height(ts, height):
    now = to_datetime(ts)
    t_diff = (now.hour*60 + now.minute)*60 + now.second
    h_diff = int(t_diff/ 7)
    new_height = int(height) - h_diff
    return new_height

def round_to_100_lower(v):
    return v - v%100

def round_to_100_upper(v):
    return v + 100 - v%100

def round_block_height(address, init_ts, init_height ):
    new_height = get_new_height(init_ts, init_height)
    #lower_height = round_to_100_lower(new_height)
    upper_height = round_to_100_upper(int(new_height)+100)
    ts, height = get_block_time(address,upper_height)
    #in case of unpredictable behavior
    save_ts, save_height = ts, height
    #get lower till prev day
    while ((to_datetime(ts).date()-to_datetime(init_ts).date()).days >= 0):
        save_ts, save_height = ts, height
        ts, height = get_block_time(address, int(height)-100)
    return save_ts, save_height


# Block/timestamp grid
def gen_blocks_daily_grid(days_ago, address):
    print('Start', address)
    block_ts = []
    #init height and ts
    ts, height = get_block_time(address)
    #dive into n days history
    for times in range(days_ago):
        not_succeed = True
        n_retries = 0
        while not_succeed:
            try:
                if n_retries > 5:
                    not_succeed = False
                    print(times, 'failed.')
                ts, height = round_block_height(address,ts, height)
                time.sleep(0.1)
                height = int(height)
                block_ts.append([ts, height])
                not_succeed = False
                print(times, 'succeed.')
                #previous day
                n_retries_next_day = 0
                not_succeed_next_day = True
                while not_succeed_next_day:
                    try:
                        if n_retries_next_day < 5:
                            prev_day_height = height - 7000
                            time.sleep(0.1)
                            ts, height = get_block_time(address, prev_day_height)
                        else:
                            break
                    except:
                        n_retries_next_day += 1
            except:
                print(times, 'failed.')
                n_retries+=1
                time.sleep(5)
    return block_ts


#grid = gen_blocks_daily_grid(
 #   (datetime.datetime.now().replace(tzinfo=pytz.UTC) - start_date).days + 1,#    # "https://lcd-1-regen.tm.p2p.org/cosmos/"
#    # "https://lcd-1-regen.tm.p2p.org"
    # f'{p2p_lcd_node}'  # https://lcd-1-regen.tm.p2p.org
#    f"{p2p_rpc_node}"
#)

In [ ]:
#Define block range for assessment period
start_block = 15375200
end_block = 17160079

# Current validators list and foundation delegations

In [ ]:
# find out active set in tendermint. REST API seems to be bugged.
tender_vals = pd.DataFrame(requests.get(f'{p2p_rpc_node}/validators?per_page=75').json()['result']['validators'])
tender_vals['pub_key'] = tender_vals['pub_key'].apply(lambda x: x['value'])
tender_vals

,address,pub_key,voting_power,proposer_priority
0,001F90AE99B52F548845DA4D93755C3D4B829097,u9Stwssek8x7wtqru3ULntBj8ExaK1fxz9Qwv/+YC9w=,123410466657,-253387777458
1,CAFE6CDB454BA42498B10CADA38A977BC0B8200C,trwfTyf0/ZQrEmaEQ0HIQ/ATYpAKGD7QZnbsrkPklHM=,78865902781,-97493638795
2,E026181D3E76C4846BCCE9443707D2EFB4604B05,dmRWr9awPjLmuwFoCr9bWmRhYLmva2EuVWxnngL8B8M=,71404782608,-345541021773
3,8ACC071A9CAB79293A12A80EF040DD0781F64DAF,8sCaQ9aButHI/pOltYdMxO8O32uzmI5nvL/EI/Zf7Y0=,60114486164,-157850392691
4,3F6B8816201A8753455E5C09906E988786B64B76,2oZ8pXUFLsp50yM7fccwF2WTQFvxfcMR/xFkZyVKJbA=,56527135159,-82991236419
5,3B2D9140742519FF28DAFF7C8CA43925117F234A,LSxGCKhYbRzRtQBtfVAS9vwwk6OgGU/dINzUw6feFwg=,55717228941,33515926671
6,04D50B5743C110B0FFAC5CA459327903712D8271,1S+ACbTEumVNXH2/UoRZ488Lnbt3NY/0yAM5scWNGkk=,54255886191,244114763560
7,FC7E26051B1773BA3E39B75019D28C36F66DED5E,oceaCqELoCYevpgttA9XXbAHx7D86Traubeg93sBa4o=,50822777459,89152175122
8,E1EB3ACEA304294DCC36111A1A910713779ADD21,QgF008QqXTdoaI/SaoGirg0FIE3NL1aa3qUlHvVc++g=,40327238518,91315967184
9,104FFCC32188359A844FD1A24E385622FC8D3698,Zg7IUTroLWX77DNJfSRql+rVIgKTSgWOV+YGVylYwOQ=,38902787541,191978400300


In [ ]:
#get list of validators
vals = cosmos_paginated_query(f'{p2p_lcd_node}/cosmos/staking/v1beta1/validators')  # LCD = REST API = only current state (not history)
vals = pd.concat([pd.DataFrame(e['validators']) for e in vals])
#extract some fields from json
vals['pub_key'] = vals['consensus_pubkey'].apply(lambda x: x['key'])
#vals = pd.merge(vals, tender_vals, on = 'pub_key', how = 'left')
vals['name'] = vals['description'].apply(lambda x: x['moniker'])
vals['delegator_shares'] = vals['delegator_shares'].astype(float)/10**6
vals['commission'] = vals['commission'].apply(lambda x: x['commission_rates']['rate']).astype(float)

#order by stake (descending)
vals = vals.sort_values('delegator_shares',ascending = False).reset_index(drop=True)
#mark active set (alternatively, we can use status field by filtering on BOND_STATUS_BONDED)
vals['is_active'] = vals['is_active'] = vals['status'].apply(lambda x: 1 if x == 'BOND_STATUS_BONDED' else 0)
#vals['voting_power'].astype(float).apply(lambda x: 1 if x >0 else 0)

Okay
Okay


In [ ]:
# Don't give stake to
## Allnodes - https://www.mintscan.io/regen/validators/regenvaloper1vmps9sn7vlgnd5latct889qyh62ulmgk72umu8
## Tavis Digital https://www.mintscan.io/regen/validators/regenvaloper1zppjyal5emta5cquje8ndkpz0rs046m7zqxrpp
## Chainode Tech Do not delegate https://www.mintscan.io/regen/validators/regenvaloper17fglrr3wlansgvrfvvtxga8s0h7unc4dzwkx4u
## Forbole regenvaloper14kn0kk33szpwus9nh8n87fjel8djx0y0c7xhe5

In [ ]:
#vals.query("operator_address == 'regenvaloper1v5hrqlv8dqgzvy0pwzqzg0gxy899rm4kgjrtdq'") # DokiaCapital
#vals.query("operator_address == 'regenvaloper14kn0kk33szpwus9nh8n87fjel8djx0y0c7xhe5'") # Forbole

In [ ]:
#current foundation delegations
f_delegs = requests.get(f'{p2p_lcd_node}/cosmos/staking/v1beta1/delegations/regen1xv7pkdjtumtvakyfgwc7daknzxrdk8yhh9kng7').json()
f_delegs = pd.DataFrame(f_delegs['delegation_responses'])
#extract some fields from json
f_delegs['operator_address'] = f_delegs['delegation'].apply(lambda x: x['validator_address'])
f_delegs['shares'] = f_delegs['balance'].apply(lambda x: x['amount']).astype(float) / 10**6

In [ ]:
#Merge validators data with foundation delegations data on validator address.
data = pd.merge(vals[['operator_address','pub_key','name','commission','is_active','delegator_shares']], f_delegs.iloc[:,[-2,-1]],
                      how='left',
        on='operator_address').fillna(0)
#save all delegations data anyway
save_data = data.copy()
#leave only active set for metrics
data = data[data['is_active']==1]#???
#data = data[~data['operator_address'].isin(
#    ['regenvaloper1v5hrqlv8dqgzvy0pwzqzg0gxy899rm4kgjrtdq',
#     'regenvaloper14kn0kk33szpwus9nh8n87fjel8djx0y0c7xhe5',
#     'regenvaloper1vmps9sn7vlgnd5latct889qyh62ulmgk72umu8',
#     'regenvaloper1zppjyal5emta5cquje8ndkpz0rs046m7zqxrpp',
#     'regenvaloper17fglrr3wlansgvrfvvtxga8s0h7unc4dzwkx4u'
#     ]
#)]
save_data.to_csv('new_val_set.csv')

In [ ]:
#data = pd.read_csv('new_val_set.csv')

# Bell Curve Factor

In [ ]:
data['cs'] =  (data['delegator_shares'] -data['shares'])/(data['delegator_shares'] -data['shares']
                                                          ).sum()
data = data.sort_values('cs')

#cummulative sum of stake for active set
cumsum = ((data['delegator_shares'] - data['shares'])/(data['delegator_shares'] - data['shares']
                                                      ).sum()).sort_values().cumsum().to_list()

#init weights
w = []
#who's left who gets 0.5
k = 5
#lower bound
l = -1
#height of the first curve
h = 0.0000001
#end height of the second curve
h2 = 0.131958
#% cum stake
p = len(list(filter(lambda x: x<0.50 ,cumsum)))
m = ((p+k)/2)*0.75
#find proper curve
A = np.array([[m**2,m,1],[p**2,p,1],[k**2,k,1]])
B = np.array([1,h2,h]).T
a,b,c=tuple(np.linalg.solve(A,B))
par = lambda x: a*x**2+b*x+c
#tail
h_tail = cumsum[p:]
for i in range(len(cumsum)):  # range(75)
    #LOW
    if i<=k:
        w.append((h-l)/k**2*i**2+l)
    #MID
    elif (i>k) & (i<=p):
        w.append(par(i))
    #HIGH
    else:
        norm = (cumsum[i] - min(h_tail))/(max(h_tail)-min(h_tail))
        w.append(h2-norm*(h2-l))
#add weights to data
data['stake_score'] = w
px.line(w)
data['t'] = data['delegator_shares'] - data['shares']
data.loc[data['name'] == 'Ekonavi', 'stake_score'] = -1.5
data.head(21).sort_values('t',ascending = False)

,operator_address,pub_key,name,commission,is_active,delegator_shares,shares,cs,stake_score,t
0,regenvaloper1aa945sl2gzeq8s7cnl8dwcvutxnqs5ca7...,u9Stwssek8x7wtqru3ULntBj8ExaK1fxz9Qwv/+YC9w=,Ekonavi,0.050000,1,12343569.987408,281087.349133,0.141885,-1.500000,12062482.638275
1,regenvaloper174tvh2dty7vsvwn2cfsmkwq8tplqgr5f3...,trwfTyf0/ZQrEmaEQ0HIQ/ATYpAKGD7QZnbsrkPklHM=,Regenerator,0.050000,1,7887379.015734,466283.044968,0.087291,-0.670640,7421095.970766
2,regenvaloper1ceunjpth8nds7sfmfd9yjmh97vxmwqfyf...,dmRWr9awPjLmuwFoCr9bWmRhYLmva2EuVWxnngL8B8M=,Simply Staking,0.100000,1,7140478.260845,256171.350531,0.080977,-0.468011,6884306.910314
3,regenvaloper1h5z08rzvrwt3pzdjc03upvuh2x0j3yskl...,8sCaQ9aButHI/pOltYdMxO8O32uzmI5nvL/EI/Zf7Y0=,Vitwit (Previously Witval),0.050000,1,6011448.616446,634624.519273,0.063245,-0.280038,5376824.097173
6,regenvaloper1tdc350ylkjfvqk4mjs6rqqksgl2wfghz2...,1S+ACbTEumVNXH2/UoRZ488Lnbt3NY/0yAM5scWNGkk=,LOA Labs,0.075000,1,5427216.621189,547926.542577,0.057393,-0.133227,4879290.078612
5,regenvaloper1exenx9cyg49y6zj84r7l29xff9eu7jx9y...,LSxGCKhYbRzRtQBtfVAS9vwwk6OgGU/dINzUw6feFwg=,Empower Validator ♻️ | 20% to plastic waste cl...,0.050000,1,5572280.121984,739449.663409,0.056846,-0.000000,4832830.458575
4,regenvaloper1t8p3zdu3h8qzggfmvvvmtdnaj4trcsfh7...,2oZ8pXUFLsp50yM7fccwF2WTQFvxfcMR/xFkZyVKJbA=,Chora Validator,0.050000,1,5653278.843795,886034.903445,0.056075,0.131958,4767243.940350
7,regenvaloper15urq2dtp9qce4fyc85m6upwm9xul3049l...,oceaCqELoCYevpgttA9XXbAHx7D86Traubeg93sBa4o=,Chorus One,0.075000,1,5082277.745993,458701.225630,0.054385,0.647828,4623576.520363
8,regenvaloper1y4dagnwxsza5tm45m3te3rpkfe74ceqvv...,QgF008QqXTdoaI/SaoGirg0FIE3NL1aa3qUlHvVc++g=,P2P.ORG - P2P Validator,0.100000,1,4032723.851888,527575.440451,0.041229,1.031066,3505148.411437
11,regenvaloper10mseqwnwtjaqfrwwp2nyrruwmjp6u5jha...,nE3MD28jzjqro3lQkcidGrADk6177iIH6oun2Q4zGCY=,Cambium,0.100000,1,3737175.114281,385858.966635,0.039420,1.281670,3351316.147646


# Governance Score

In [ ]:
#Query all proposals
proposals = requests.get(f'{p2p_lcd_node}/cosmos/gov/v1/proposals').json()
proposals = pd.DataFrame(proposals['proposals']).rename(columns={"id": "proposal_id"})
proposals['voting_end_time'] = pd.to_datetime(proposals['voting_end_time'])

#filter on status and date
proposals = proposals[proposals['status'].isin([
    'PROPOSAL_STATUS_PASSED', 'PROPOSAL_STATUS_REJECTED'
]) &
(proposals['voting_end_time'] >= end_date - datetime.timedelta(days=180))
]
proposals = proposals[['proposal_id','voting_end_time']]
proposals['voting_end_time'] = pd.to_datetime(proposals['voting_end_time'])



In [ ]:
t_ = pd.read_csv('votes.csv')
account_addresses = t_[['operator_address','account_address']]
#add account addresses to data
data = pd.merge(data, account_addresses, on = 'operator_address', how = 'left')
data.drop_duplicates(inplace=True)

# Val creation Txs

In [ ]:
accounts = data['account_address']
votes = t_[['account_address','proposal_id']]
votes['proposal_id'] = votes['proposal_id'].astype(int)
votes

,account_address,proposal_id
0,regen1aa945sl2gzeq8s7cnl8dwcvutxnqs5cazdugze,44
1,regen1aa945sl2gzeq8s7cnl8dwcvutxnqs5cazdugze,43
2,regen1aa945sl2gzeq8s7cnl8dwcvutxnqs5cazdugze,41
3,regen1aa945sl2gzeq8s7cnl8dwcvutxnqs5cazdugze,40
4,regen1aa945sl2gzeq8s7cnl8dwcvutxnqs5cazdugze,39
...,...,...
534,regen1wm7d4285myyd276zcnhex73eyhqh0h7jtgt4n7,25
535,regen1wm7d4285myyd276zcnhex73eyhqh0h7jtgt4n7,24
536,regen1wm7d4285myyd276zcnhex73eyhqh0h7jtgt4n7,22
537,regen1wm7d4285myyd276zcnhex73eyhqh0h7jtgt4n7,21


In [ ]:
def query_creation_tx(operator_address):
    url = f"{p2p_lcd_node}/cosmos/tx/v1beta1/txs?events=create_validator.validator%3D'{operator_address}'"
    return requests.get(url).json()
#creation_txs = []
#for oper in tqdm(data['operator_address']):
#    creation_txs.append([query_creation_tx(oper),oper])
#new_missing_creation_times = [e[1] for e in creation_txs if len(e[0]['txs']) != 0]
#len(new_missing_creation_times)

#manual creation tx gathering:
creation_times = pd.DataFrame([
    ['regenvaloper1mpy34essw3hawylg4k393m08nnvsfv6ag2dcqn','2022-10-07 12:05:05'],
    ['regenvaloper1rx38r50pjccn7w7kn4qyqcd2zcgqnx4cnh8xnq','2022-10-19 20:48:01'],
    ['regenvaloper1407d25jg9p565nrexr8ygtaz3smuh63qg597a4','2021-08-07 07:35:34'],
    ['regenvaloper1gjvu75cq6qxyrtdv66lx9xe92jw9gqdetagaze','2021-06-25 08:47:43'],
    ['regenvaloper1t8p3zdu3h8qzggfmvvvmtdnaj4trcsfh79xp4e','2022-12-05 03:07:12'],
    ['regenvaloper1vnxgn4uwtr0ug8z3u07c26chxn5a95lggwxtzv','2021-07-31 00:29:03'],
    ['regenvaloper1tkcu62sr47qy3gyh2kwy448q649nhqafuzgxwk','2021-08-10 18:44:31'],
    ['regenvaloper1d92tfnkge6qcvt3jy42j0dx6vvt4tjmfldmhue','2021-07-31 10:36:14'],
    ['regenvaloper1vn6sachfy4aacyl5gwc4ykkn6kaqsqv3vzjd8t','2021-08-03 20:25:44'],
    ['regenvaloper1a5asvn8sk3zqf4qjctgkqynzgqx57ug79r9aqt','2021-04-30 21:06:15'],
    ['regenvaloper16fzkfglaqmp36cvqum0zguyanhmtkj89pdt3wz','2021-04-15 18:22:46'],
    ['regenvaloper1vmps9sn7vlgnd5latct889qyh62ulmgk72umu8','2022-01-22 17:58:13'],
    ['regenvaloper1zmjm2cujkvf6xpwlxtx2uqja5rgln7dxs3naz9','2022-03-21 16:25:34'],
    ['regenvaloper1exenx9cyg49y6zj84r7l29xff9eu7jx9ygwnv5','2022-10-17 18:46:00'],
    ['regenvaloper15t7cmpmlgzllvs2wv4zly3lhsd0yd02ktppmsm','2022-02-14 12:32:12'],
    ['regenvaloper1wu92z5y3sk0rnywenqudv69eqsq0gr4unhq04g','2022-05-03 11:19:37'],
    ['regenvaloper1vys9dreue4e8xrga2zmuzth4z4gkmmgvjdxfdz','2021-08-29 15:42:05'],
    ['regenvaloper1wm7d4285myyd276zcnhex73eyhqh0h7jh0kst0','2022-11-18 22:19:29'],
    ['regenvaloper15khamenky6kmmv8npd0mnwme22fhks735zdpk6','2021-10-11 12:41:38'],
    ['regenvaloper1c4y3j05qx652rnxm5mg4yesqdkmhz2f6dl7hhk','2022-03-22 23:29:32'],
    ['regenvaloper1083svrca4t350mphfv9x45wq9asrs60ct2p9a5','2022-03-22 23:29:32'],
    ['regenvaloper10mseqwnwtjaqfrwwp2nyrruwmjp6u5jhah4c3y','2021-09-14 21:17:04'],
    ['regenvaloper15qepg86xxesg7k633ent5t9vaweyf5ztgw2pyf','2022-01-28 15:36:31'],
    ['regenvaloper1aa945sl2gzeq8s7cnl8dwcvutxnqs5ca72pd6g','2021-08-09 05:24:30'],
    ['regenvaloper1ss2f0nl7sn42x8x0d337mj9welzml8h0f5erue','2021-10-20 13:51:51'],
    ['regenvaloper1uccl5ugxrm7vqlzwqr04pjd320d2fz0z3hc6vm','2021-09-16 13:34:46'],
    ['regenvaloper1vksj7nqkd89uvc4kh8f5mh4f3235megmcgjfmq','2021-10-28 10:51:42'],
    ['regenvaloper1ldgt9pky839sr925jnrqr4kxzh5h3yhlth3jz0','2021-08-05 21:39:11'],
    ['regenvaloper1tdc350ylkjfvqk4mjs6rqqksgl2wfghz2fx95h','2021-04-17 05:40:56'],
    ['regenvaloper139el73jc3jca4atp7rvgc6lupk2na6rqhl4r9f','2023-04-14 13:20:29'],
    ['regenvaloper19geuzlxa7mvlagensfehgdeuwhcjyeamdfexlh','2023-03-05 06:26:31'],
    ['regenvaloper1a023gshv0a9pjvl50wqwsuqmj8pfmtj4hpk9pw','2023-05-14 02:24:10'],
    ['regenvaloper1mvvypwrdpr4esn6g4rz7upj9wd4szrgl65qvwa','2023-06-08 12:22:37'],
    ['regenvaloper1gfy39gctfrfey9nfky3d2sc09erph4c0mzsmwr', '2021-04-15 19:34:24' ],
    ['regenvaloper1ra6p2sy9hnrvrvtmkjkfwh7yfzpqpv0cmdeuaf','2023-02-12 22:25:28'],
    ['regenvaloper1aymx97z6eanltjg8cwarf6hrmasmnvquhzdcr4', '2023-08-27 11:25:12'],
    ['regenvaloper1adzpsnvegat0mdcd2yn3hpy8hly9t20cwzjmj6', '2024-01-23 13:53:24'], # STAVR
    ['regenvaloper105g89nqllu33nend0ce5eup4zxn0d4kfr2v7w8', '2024-03-11 04:44:31'], # polkachu.com
    ['regenvaloper1gpeunhcmw6q82l9zkf503l6l3vsfv7wae03rav', '2024-02-09 11:13:18'], # Oldcat
    ['regenvaloper1hsafk3cfrdcn2qjar7awz68qhmlj6tz2qhz67p', '2024-02-26 00:49:43'], # DECALI.io
    ['regenvaloper1wexza7kxc8qktxqcwrt65anwuxjw8pg0akx36t', '2024-03-26 18:42:20'], # Earthist

    ['regenvaloper15m4wx9nftlenp77gzmjspl5n3vqql53k3uvxde', '2024-05-13 23:16:36'], # Quebec
    ['regenvaloper17uq9hjk5vpglnv2aexnezu0tqw2e76xen5qdch', '2024-06-12 20:46:38'], # SyaNodes
    ['regenvaloper18lyskamafpdqt6yxrj3tj346zun6nkyu7hq0yr', '2024-06-19 20:44:02'], # Shazoe
    ['regenvaloper1th6g4q63ee0x3wr5tctrxsh3lugj2up6fj64rr', '2024-04-15 01:23:56'], # skynodejs

    ['regenvaloper1en2ntpqeuvv7pqz2488rwl5aq8um4yukltkp6u', '2024-06-06 23:16:36'], # Kaiden
    ['regenvaloper1ft2g6kcuavxey8xhz0g6zrhzwpr8lse82prm95', '2024-06-19 20:46:38'], # Shamura
    ['regenvaloper1hkpgalxpukzqsmklj9463lqjz492f9yvc8y8ee', '2024-05-25 20:44:02'], # chainpiper
    ['regenvaloper1nkgf2pfumtnlpgr5s6340kr9l7g4pkt7dzev6l', '2024-06-06 01:23:56'], # Razor
    ['regenvaloper1scs85vk838wz7e3km0cfzdhexq6mxfneh4jfpz', '2024-06-19 01:23:56'], # skynodejs
],
    columns = ['operator_address','creation_ts']
)
#set(new_missing_creation_times) - (set(creation_times['operator_address']))

In [ ]:
creation_times = pd.merge(data[['account_address','operator_address']], creation_times, how = 'left', on = 'operator_address').fillna('2021-01-01 00:00:00')
creation_times['creation_ts'] = pd.to_datetime(creation_times['creation_ts'])
proposals['proposal_id'] = proposals['proposal_id'].astype(int)
votes = pd.merge(votes, proposals, on= 'proposal_id', how = 'left')
votes = pd.merge(votes, creation_times, how = 'left', on = 'account_address')
votes['voting_end_time'] = pd.to_datetime(votes['voting_end_time'].astype(str).str[:18])
curr_votes = votes[votes['voting_end_time']>=votes['creation_ts']]

def count_max_proposals(x):
    res = len(
        proposals[pd.to_datetime(proposals['voting_end_time'].astype(str).str[:18]) >= x]
    )
    return res

gov = curr_votes.groupby('operator_address',as_index=False).agg({'proposal_id':'count','creation_ts':'min'})
gov['max_proposals'] = gov['creation_ts'].apply(lambda x: count_max_proposals(x))
gov['gov_score'] = gov['proposal_id']/gov['max_proposals']
gov

,operator_address,proposal_id,creation_ts,max_proposals,gov_score
0,regenvaloper10mseqwnwtjaqfrwwp2nyrruwmjp6u5jha...,2,2021-09-14 21:17:04,3,0.666667
1,regenvaloper139el73jc3jca4atp7rvgc6lupk2na6rqh...,1,2023-04-14 13:20:29,3,0.333333
2,regenvaloper15urq2dtp9qce4fyc85m6upwm9xul3049l...,2,2021-01-01 00:00:00,3,0.666667
3,regenvaloper174tvh2dty7vsvwn2cfsmkwq8tplqgr5f3...,2,2021-01-01 00:00:00,3,0.666667
4,regenvaloper1aa945sl2gzeq8s7cnl8dwcvutxnqs5ca7...,2,2021-08-09 05:24:30,3,0.666667
5,regenvaloper1c4y3j05qx652rnxm5mg4yesqdkmhz2f6d...,2,2022-03-22 23:29:32,3,0.666667
6,regenvaloper1ceunjpth8nds7sfmfd9yjmh97vxmwqfyf...,2,2021-01-01 00:00:00,3,0.666667
7,regenvaloper1exenx9cyg49y6zj84r7l29xff9eu7jx9y...,2,2022-10-17 18:46:00,3,0.666667
8,regenvaloper1h5z08rzvrwt3pzdjc03upvuh2x0j3yskl...,2,2021-01-01 00:00:00,3,0.666667
9,regenvaloper1kl83t6gm2y5lgg5c5h3sz87tt5fg3cmyl...,1,2021-01-01 00:00:00,3,0.333333


# Sum up Gov Score and add it to data

In [ ]:
gov.to_csv('Governance_'+report_date+'.csv')
gov = pd.read_csv('Governance_'+report_date+'.csv').iloc[:,1:]
gov = gov[['operator_address','gov_score']]
data = pd.merge(data, gov[['operator_address','gov_score']], on = 'operator_address', how = 'left')
gov

,operator_address,gov_score
0,regenvaloper10mseqwnwtjaqfrwwp2nyrruwmjp6u5jha...,0.666667
1,regenvaloper139el73jc3jca4atp7rvgc6lupk2na6rqh...,0.333333
2,regenvaloper15urq2dtp9qce4fyc85m6upwm9xul3049l...,0.666667
3,regenvaloper174tvh2dty7vsvwn2cfsmkwq8tplqgr5f3...,0.666667
4,regenvaloper1aa945sl2gzeq8s7cnl8dwcvutxnqs5ca7...,0.666667
5,regenvaloper1c4y3j05qx652rnxm5mg4yesqdkmhz2f6d...,0.666667
6,regenvaloper1ceunjpth8nds7sfmfd9yjmh97vxmwqfyf...,0.666667
7,regenvaloper1exenx9cyg49y6zj84r7l29xff9eu7jx9y...,0.666667
8,regenvaloper1h5z08rzvrwt3pzdjc03upvuh2x0j3yskl...,0.666667
9,regenvaloper1kl83t6gm2y5lgg5c5h3sz87tt5fg3cmyl...,0.333333


# Jails

In [ ]:
#query valcons address to create valcons/valoper mapping
valcons = requests.get(f'{p2p_lcd_node}/cosmos/base/tendermint/v1beta1/validatorsets/latest').json()
valcons = pd.DataFrame(valcons['validators'])
valcons['pub_key'] = valcons['pub_key'].apply(lambda x: x['key'])
valcons['valcons_address'] = valcons['address']
valcons = pd.merge(data[['operator_address','pub_key']],
                   valcons[['valcons_address','pub_key']],
                   on = 'pub_key', how = 'left')

jails = cosmos_paginated_query(f'{p2p_lcd_node}/cosmos/slashing/v1beta1/signing_infos')
jails = pd.concat([pd.DataFrame(e['info']).rename(columns = {'address':'valcons_address'})
                   for e in jails])
jails = jails[~jails['tombstoned']]

Okay
Okay


In [ ]:
#check if anyone was jailed last month
jails['jail_date'] =  pd.to_datetime(jails['jailed_until'], format='mixed').apply(lambda x: x.date())
jails['jailed_last_1m'] = (jails['jail_date'] + datetime.timedelta(days = 30)
                           > datetime.datetime(2024,4,1,0,0,0,0).date()).astype(int)
jails.to_csv('Jails_'+report_date+'.csv')
jails = pd.merge(valcons[['operator_address','valcons_address']], jails, on = 'valcons_address', how = 'right')
jails = jails[~jails['valcons_address'].isna()]

#load previous jails info
prev_jails = pd.read_csv('Jails_'+report_date+'.csv').iloc[:,1:].rename(columns = {'address':'valcons_address'})
prev_jails['prev_jail_date'] =  pd.to_datetime(prev_jails['jailed_until'], format='mixed').apply(lambda x: x.date())

Jails = pd.merge(jails[['operator_address','valcons_address','jail_date']],
                 prev_jails[['valcons_address','prev_jail_date']],
                 on = 'valcons_address', how = 'left'
                 ).reset_index(drop=True)

#calculate jail score
Jails['jail_score'] = 1.0
for i in range(len(Jails)):
    jailed_1m = pd.to_datetime(Jails['jail_date'][i] + datetime.timedelta(days = 30), utc=True) > pd.to_datetime(start_date)
    if (Jails['prev_jail_date'][i] != Jails['jail_date'][i]) & (~isinstance(Jails['prev_jail_date'][i],float)):
        prev_jailed_6m = (Jails['prev_jail_date'][i] + datetime.timedelta(days = 180)) > pd.to_datetime('2023-07-01')
        if jailed_1m & prev_jailed_6m:
            print(i)
            Jails['jail_score'][i] = 0
        else:
            Jails['jail_score'][i] = 0.5
            continue
    else:
        if jailed_1m:
            Jails['jail_score'][i] = 0.5
Jails.sort_values('jail_score')

data = pd.merge(data, Jails[['operator_address','jail_score']], on = 'operator_address', how = 'left')

# Ecoscore

In [ ]:
#Ecoscore is filled manually
data['eco_score'] = 0
data.loc[data['name'].isin(['Stakin','Stakely.io','Regenerator']),'eco_score'] = 1

# Community Score

In [ ]:
#read community score file
Community = pd.read_csv('Regen foundation delegations - regen_foundation_delegations_2.csv')[['operator_address','community_score']]
#add to data
data = pd.merge(data,Community, on = 'operator_address', how = 'left')
data['community_score'] = data['community_score'].fillna(0.3)

In [ ]:
data.to_csv('test_.csv',index=False)

# Uptime Take Consensus key from stake_data and merge on that

In [ ]:
#get list of validators with hex addresses (contains only active set)

# hex_vals = requests.get(f'{p2p_lcd_node}/validators?per_page=150').json()
hex_vals = requests.get(f'https://rpc-regen.ecostake.com/validators?per_page=150').json()
hex_vals = pd.DataFrame(hex_vals['result']['validators'])
hex_vals['pub_key'] = hex_vals['pub_key'].apply(lambda x: x['value'])
hex_vals = hex_vals[['pub_key','address','voting_power']]

#create hex/operator addresses mapping by merging on consensus pubkey
data = pd.merge(data, hex_vals, how = 'outer', on = 'pub_key' )

#get rid of non-active validators
val_map = data[['operator_address','address']].copy()
val_map = val_map[~val_map['address'].isna()]

#check whether number of active validators is 75 (there's no missalingment in hex/valoper mappping)
val_map = dict(zip(val_map['address'],val_map['operator_address']))
print(len(val_map))
#set of hex addresses
vals_set = set(val_map.keys())

21


In [ ]:
up_ = pd.read_csv('D:\\t_.csv')

In [ ]:
import json
import re

up_['block_number'] = up_.apply(lambda x: int(re.search(r"'height': '(\d+)'", x['block']).group(1)), axis = 1)

In [ ]:
#function that queries blocks and checks whether every validator from a given set participated
#either in validation
#or in block proposition
#Params:
# start_block - the last block we will query
# depth - n blocks before the last one we will query
#The output: the dict with hex addresses and total numbers of blocks they participated in
import random
rpc = [
    'http://public-rpc.regen.vitwit.com:26657',
    'https://rpc-regen.ecostake.com',
    'https://regen-rpc.w3coins.io'
]
d = 0
def check_uptime_parallel(start_block, depth, val_map, n_jobs):
    start = datetime.datetime.now()
    vals_uptime = dict(zip(val_map.values(),[depth for i in range(len(val_map))]))


    h = start_block-i
    #block = requests.get(f'{p2p_rpc_node}/block?height='+str(h)).json()
    #d = block
   # print(h)
    try:
        block = json.loads(up_[up_['block_number']==h]['block'].values[0].replace("'", '"').replace('None','"0"'))
        sigs = block['last_commit']['signatures']
        proposer = [block['header']['proposer_address']]
        vals_sigs = set([s['validator_address'] for s in sigs]+proposer)
        failed_ = vals_set - vals_sigs
        for failed in failed_:
            vals_uptime[val_map[failed]]-=1
        end = datetime.datetime.now()

        return vals_uptime
    except Exception as e:
        print(e,h)
        return 0




      #  return failed

   # failed_bucket = Parallel(n_jobs=n_jobs)(delayed(process)(i) for i in (range(depth)))



start_block = 17161142
end_block = 0
#init params for uptime computations
#numbers container
vals_uptime = dict(zip(val_map.values(),[0 for i in range(len(val_map))]))
#total amount blocks processed
cumm_depth = 0

#size of depth chunk we want to process (more -> faster, but error prone)
depth = 1
# iters = (start_block - end_block)/depth
iters  = 99999



for chunk_n in tqdm(range(int(iters)+1)):
    chunk_start_block = start_block - chunk_n*depth
    is_not_succeed = True
    n_retries = 0
    while is_not_succeed and (n_retries < 10):

        chunk = check_uptime_parallel(chunk_start_block, depth, val_map, 5)
        is_not_succeed = False
    if chunk == 0: continue
    if n_retries == 10:
        break
    for val in chunk.keys():
        vals_uptime[val]+=chunk[val]
    cumm_depth+=depth
for key in vals_uptime.keys():
    vals_uptime[key]/=cumm_depth

Uptime = pd.DataFrame(index = vals_uptime.keys(), data=vals_uptime.values()).reset_index()
Uptime.columns = ['operator_address','uptime']
Uptime.to_csv('Uptime_'+report_date+'.csv')
data = pd.merge(data, Uptime, on = 'operator_address', how = 'left')
data['uptime_score'] = ((data['uptime']-0.98)/(1-0.98)).apply(lambda x: 0 if x<0 else x)


data = data[~data['operator_address'].isna()]#[['name','uptime']].sort_values('uptime')
data['uptime'] = data['uptime_score'].fillna(0)

  1%|          | 1003/100000 [00:00<00:21, 4575.68it/s]

Expecting ',' delimiter: line 1 column 2079 (char 2078) 17160511


  4%|▍         | 4388/100000 [00:01<00:23, 4124.05it/s]

Expecting ',' delimiter: line 1 column 8025 (char 8024) 17157268


  7%|▋         | 7341/100000 [00:01<00:22, 4120.18it/s]

...

Expecting ',' delimiter: line 1 column 15043 (char 15042) 17069077


 98%|█████████▊| 98435/100000 [00:22<00:00, 4273.81it/s]

Expecting ',' delimiter: line 1 column 13760 (char 13759) 17063474


100%|██████████| 100000/100000 [00:23<00:00, 4305.22it/s]

index 0 is out of bounds for axis 0 with size 0 17061141
index 0 is out of bounds for axis 0 with size 0 17061140
index 0 is out of bounds for axis 0 with size 0 17061139
index 0 is out of bounds for axis 0 with size 0 17061138
index 0 is out of bounds for axis 0 with size 0 17061137
index 0 is out of bounds for axis 0 with size 0 17061136
index 0 is out of bounds for axis 0 with size 0 17061135
index 0 is out of bounds for axis 0 with size 0 17061134
index 0 is out of bounds for axis 0 with size 0 17061133
index 0 is out of bounds for axis 0 with size 0 17061132
index 0 is out of bounds for axis 0 with size 0 17061131
index 0 is out of bounds for axis 0 with size 0 17061130
index 0 is out of bounds for axis 0 with size 0 17061129
index 0 is out of bounds for axis 0 with size 0 17061128
index 0 is out of bounds for axis 0 with size 0 17061127
index 0 is out of bounds for axis 0 with size 0 17061126
index 0 is out of bounds for axis 0 with size 0 17061125
index 0 is out of bounds for ax

In [ ]:
pd.DataFrame(index = vals_uptime.keys(), data=vals_uptime.values()).reset_index()

,index,0
0,regenvaloper139el73jc3jca4atp7rvgc6lupk2na6rqh...,0.992749
1,regenvaloper1zmjm2cujkvf6xpwlxtx2uqja5rgln7dxs...,0.997967
2,regenvaloper1wm7d4285myyd276zcnhex73eyhqh0h7jh...,0.964023
3,regenvaloper1kl83t6gm2y5lgg5c5h3sz87tt5fg3cmyl...,0.996585
4,regenvaloper1wexza7kxc8qktxqcwrt65anwuxjw8pg0a...,0.994632
5,regenvaloper1aymx97z6eanltjg8cwarf6hrmasmnvquh...,0.995433
6,regenvaloper1vys9dreue4e8xrga2zmuzth4z4gkmmgvj...,0.999149
7,regenvaloper1n3mhyp9fvcmuu8l0q8qvjy07x0rql8q4u...,0.996234
8,regenvaloper1c4y3j05qx652rnxm5mg4yesqdkmhz2f6d...,0.999409
9,regenvaloper1snn4uhxh04gzpgk4l8naw3n6fu7ucwx37...,0.999920


# Final Scoring

In [ ]:
data.to_csv('data_'+report_date+'.csv')
data['gov_score'] = data['gov_score'].fillna(0)
data = data.drop_duplicates()
score = data[['operator_address','name','delegator_shares','shares'] +
             [col for col in data.columns if col.find('score')>-1] + ['commission']]
score['commission'] = score['commission'].replace(0, 0.05)

score['before_jailing'] = (score['stake_score']*0.35 +
                           score['uptime_score']*0.225 +
                           score['gov_score']*0.125 +
                           score['community_score']*0.225 +
                           score['eco_score']*0.075
                           )

#set malicious guys to zero
score.loc[score['operator_address'].isin(
    ['regenvaloper1vmps9sn7vlgnd5latct889qyh62ulmgk72umu8',
     'regenvaloper1407d25jg9p565nrexr8ygtaz3smuh63qg597a4',
     'regenvaloper1zppjyal5emta5cquje8ndkpz0rs046m7zqxrpp'
     ]
), 'before_jailing'] = 0


score['before_comm'] = score['before_jailing'].fillna(0)*data['jail_score']
score['base'] = ((score['before_comm']/score['before_comm'].sum())).apply(lambda x: 0 if x < 0 else x)
score['base_priced'] = score['base']/score['commission']
score['base_adj'] = (score['base_priced']/score['base_priced'].sum())
score['deleg_shares_wo_f'] = score['delegator_shares'] - score['shares']

In [ ]:
score

,operator_address,name,delegator_shares,shares,stake_score,gov_score,jail_score,eco_score,community_score,uptime_score,commission,before_jailing,before_comm,base,base_priced,base_adj,deleg_shares_wo_f
0,regenvaloper139el73jc3jca4atp7rvgc6lupk2na6rqh...,w3coins,3072184.847199,2334200.206390,-1.000000,0.333333,1.000000,0,0.300000,0.637427,0.020000,-0.097412,-0.097412,0.000000,0.000000,0.000000,737984.640809
1,regenvaloper1zmjm2cujkvf6xpwlxtx2uqja5rgln7dxs...,Keplr,2577847.383119,377121.291239,-0.960000,0.333333,1.000000,0,0.300000,0.898339,0.100000,-0.024707,-0.024707,0.000000,0.000000,0.000000,2200726.091880
2,regenvaloper1wm7d4285myyd276zcnhex73eyhqh0h7jh...,Arcane Forge,2616322.270711,372556.716330,-0.840000,0.666667,0.500000,0,1.000000,0.000000,0.100000,0.014333,0.007167,0.000766,0.007665,0.000517,2243765.554381
3,regenvaloper1kl83t6gm2y5lgg5c5h3sz87tt5fg3cmyl...,Informal Systems,2911890.335132,644757.881123,-0.640000,0.333333,1.000000,0,1.000000,0.829230,0.081100,0.229243,0.229243,0.024517,0.302304,0.020396,2267132.454009
4,regenvaloper1wexza7kxc8qktxqcwrt65anwuxjw8pg0a...,Earthist,2718340.663656,304555.145647,-0.360000,0.500000,1.000000,0,0.300000,0.731576,0.090000,0.168605,0.168605,0.018032,0.200353,0.013517,2413785.518009
5,regenvaloper1aymx97z6eanltjg8cwarf6hrmasmnvquh...,Commons Hub,2926709.560000,375521.336460,0.000000,0.000000,1.000000,0,0.300000,0.771639,0.100000,0.241119,0.241119,0.025787,0.257869,0.017398,2551188.223540
6,regenvaloper1vys9dreue4e8xrga2zmuzth4z4gkmmgvj...,Stakecito,3619494.727293,868771.064335,0.545194,0.666667,1.000000,0,0.300000,0.957433,0.050000,0.557074,0.557074,0.059577,1.191546,0.080391,2750723.662958
7,regenvaloper1n3mhyp9fvcmuu8l0q8qvjy07x0rql8q4u...,0base.vc,3752713.090976,848577.139134,0.957756,0.666667,1.000000,0,0.300000,0.811702,0.050000,0.668681,0.668681,0.071513,1.430267,0.096497,2904135.951842
8,regenvaloper1c4y3j05qx652rnxm5mg4yesqdkmhz2f6d...,ECO Stake 🌱,3890278.754180,871257.258758,1.237684,0.666667,1.000000,0,0.400000,0.970453,0.050000,0.824875,0.824875,0.088218,1.764356,0.119037,3019021.495422
9,regenvaloper1snn4uhxh04gzpgk4l8naw3n6fu7ucwx37...,Chainflow,3672437.143491,626377.015610,1.384979,0.333333,1.000000,0,1.000000,0.995994,0.080000,0.975508,0.975508,0.104328,1.304094,0.087984,3046060.127881


# Report Generation

In [ ]:
#Visualisation of scores

v = score.copy().reset_index().sort_values('deleg_shares_wo_f')
fig = make_subplots(rows = 6, cols = 1,
                    subplot_titles = ['Stake distr. wo foundation',
                                      'Foundation Stake distribution',
                                     'Bell Curve Factor Score',
                                     'Uptime Score',
                                     'Governance Score',
                                     'Community Score'],
                                       )
fig.add_trace(go.Bar(x=v['name'],y=v['base'], name = 'before comm adjustment'), row = 2, col=1)
fig.add_trace(go.Bar(x=v['name'],y=v['base_adj'], name = 'new, adjusted on comm'), row = 2, col=1)
#fig.add_trace(go.Bar(x=v['name'],y=v['base'], name = 'new, before adjustment'))
fig.add_trace(go.Bar(x=v['name'],y=v['shares']/v['shares'].sum(), name = 'old'), row = 2, col=1)
fig.add_trace(go.Bar(x=v['name'],y=v['delegator_shares']-v['shares'],name = 'Stake WO foundation'), row = 1, col=1)

fig.add_trace(go.Bar(x=v['name'],y=v['stake_score'], name = 'Bell Curve Factor Score'), row = 3, col=1)

fig.add_trace(go.Bar(x=v['name'],y=v['uptime_score'], name = 'Uptime Score'), row = 4, col=1)

fig.add_trace(go.Bar(x=v['name'],y=v['gov_score'], name = 'Governance Score'), row = 5, col=1)

fig.add_trace(go.Bar(x=v['name'],y=v['community_score'], name = 'Community Score'), row = 6, col=1)

fig.update_layout(width = 2000, height = 2000)
fig.show()
fig.write_html('scoring_result_'+report_date+'.html')

In [ ]:
# Distribute 37 mln - point 5 in task
final = score.sort_values('delegator_shares',ascending = False)
# final['new_delegations_distr'] = final['base_adj']*29.5*10**6
final['new_delegations_distr'] = final['base_adj']*37*10**6  # Общая сумма - 37M REGEN

# redelegations data considering active set
r_data = pd.merge(save_data, final[['operator_address','new_delegations_distr']],
                  on ='operator_address',
                  how = 'left')[['operator_address','shares','new_delegations_distr']]
r_data['redeleg'] = r_data['new_delegations_distr'].fillna(0) - r_data['shares']

redeleg_state = r_data[['operator_address','redeleg']].fillna(0).sort_values('redeleg')
plus = redeleg_state[redeleg_state['redeleg']>0].sort_values('redeleg', ascending = False).drop_duplicates()
minus = redeleg_state[redeleg_state['redeleg']<0].sort_values('redeleg', ascending = False).drop_duplicates()
redeleg_tx = []
for i in range(len(minus)):
    available_redeleg = -minus.iloc[i,1]
    val_from = minus.iloc[i,0]
    for j in range(len(plus)):
        required_redeleg = plus.iloc[j,1]
        val_to = plus.iloc[j,0]
        if required_redeleg <= available_redeleg:
            fact_redeleg = required_redeleg
            redeleg_tx.append(['redelegate',val_from,val_to,fact_redeleg])
            available_redeleg -= required_redeleg
            plus.iloc[j,1] = 0
        else:
            fact_redeleg = available_redeleg
            plus.iloc[j,1] -= fact_redeleg
            redeleg_tx.append(['redelegate',val_from,val_to,fact_redeleg])
            available_redeleg = 0
        if available_redeleg == 0:
            minus.iloc[i,1] = 0
            break
left_to_deleg = plus[plus['redeleg']>0].reset_index(drop=True)
for i in range(len(left_to_deleg)):
    redeleg_tx.append(['delegate','',left_to_deleg.iloc[i,0],left_to_deleg.iloc[i,1]])

print(left_to_deleg.sum())

redeleg_tx_df = pd.DataFrame(redeleg_tx, columns = ['action','from','to','amount']).sort_values('amount')
redeleg_tx_df = redeleg_tx_df[redeleg_tx_df['amount']>1]
redeleg_tx_df['amount'] = redeleg_tx_df['amount'].apply(lambda x: round(x,6))
redeleg_tx_df.to_csv('redeleg_tx_'+report_date+'.csv', sep = ',')

operator_address    regenvaloper1kl83t6gm2y5lgg5c5h3sz87tt5fg3cmyl...
redeleg                                                    648.485950
dtype: object


In [ ]:
final = final[['operator_address', 'name', 'delegator_shares', 'shares', 'stake_score',
       'uptime_score', 'jail_score', 'gov_score',  'community_score','eco_score','commission',
               'base', 'base_adj','new_delegations_distr']]
final.columns = ['operator_address', 'name', 'current_stake', 'foundation_stake', 'stake_score',
       'uptime_score', 'jail_score', 'gov_score',  'community_score','eco_score','commission',
       'distr_before_comm_adj', 'new_distribution','new_foundation_stake']
final['new_foundation_stake']= final['new_foundation_stake'].fillna(0)
final.to_csv('regen_foundation_delegations_'+report_date + '.csv')

In [ ]:
final.drop_duplicates()

,operator_address,name,current_stake,foundation_stake,stake_score,uptime_score,jail_score,gov_score,community_score,eco_score,commission,distr_before_comm_adj,new_distribution,new_foundation_stake
20,regenvaloper1aa945sl2gzeq8s7cnl8dwcvutxnqs5ca7...,Ekonavi,12343569.987408,281087.349133,-1.500000,0.988983,1.000000,0.666667,1.000000,0,0.050000,0.000626,0.000845,31259.291932
19,regenvaloper174tvh2dty7vsvwn2cfsmkwq8tplqgr5f3...,Regenerator,7887379.015734,466283.044968,-0.670640,0.989483,1.000000,0.666667,0.700000,1,0.050000,0.032484,0.043833,1621817.374808
18,regenvaloper1ceunjpth8nds7sfmfd9yjmh97vxmwqfyf...,Simply Staking,7140478.260845,256171.350531,-0.468011,0.941908,1.000000,0.666667,0.900000,0,0.100000,0.035716,0.024097,891576.498604
17,regenvaloper1h5z08rzvrwt3pzdjc03upvuh2x0j3yskl...,Vitwit (Previously Witval),6011448.616446,634624.519273,-0.280038,0.925883,1.000000,0.666667,1.000000,0,0.050000,0.044773,0.060414,2235321.647433
14,regenvaloper1t8p3zdu3h8qzggfmvvvmtdnaj4trcsfh7...,Chora Validator,5653278.843795,886034.903445,0.131958,0.798181,1.000000,0.666667,1.000000,0,0.050000,0.057121,0.077077,2851843.702875
15,regenvaloper1exenx9cyg49y6zj84r7l29xff9eu7jx9y...,Empower Validator ♻️ | 20% to plastic waste cl...,5572280.121984,739449.663409,-0.000000,1.000000,1.000000,0.666667,0.300000,0,0.050000,0.040194,0.054236,2006738.571951
16,regenvaloper1tdc350ylkjfvqk4mjs6rqqksgl2wfghz2...,LOA Labs,5427216.621189,547926.542577,-0.133227,0.873300,1.000000,0.666667,1.000000,0,0.075000,0.049003,0.044081,1631007.669839
13,regenvaloper15urq2dtp9qce4fyc85m6upwm9xul3049l...,Chorus One,5082277.745993,458701.225630,0.647828,0.875303,1.000000,0.666667,0.800000,0,0.075000,0.073474,0.066095,2445520.815211
12,regenvaloper1y4dagnwxsza5tm45m3te3rpkfe74ceqvv...,P2P.ORG - P2P Validator,4032723.851888,527575.440451,1.031066,1.000000,1.000000,0.333333,1.000000,0,0.100000,0.091176,0.061515,2276040.996203
8,regenvaloper1c4y3j05qx652rnxm5mg4yesqdkmhz2f6d...,ECO Stake 🌱,3890278.754180,871257.258758,1.237684,0.970453,1.000000,0.666667,0.400000,0,0.050000,0.088218,0.119037,4404366.622475
